In [1]:
import json
import shutil
from tqdm import tqdm
import soundfile as sf
import json
import numpy as np

def clip(data, thres):
    return np.maximum(np.minimum(thres,data),-thres)

def coclip(data,thres):
    return data - clip(data,thres)

def normsq(data):
    return np.sum(data*data)

def snr(data, thres):
    # print(normsq(data),normsq(coclip(data,thres)))
    return 10 * np.log10( normsq(data) / (normsq(coclip(data,thres))+1e-7) )

def determine_threshold(data,target_snr):
    low=0.0
    high=0.999
    mid = (low+high)/2
    current_snr = snr(data, mid)
    while np.abs(target_snr - current_snr) > 1e-4:
        if current_snr < target_snr:
            low = mid
        else:
            high = mid
        mid = (low+high)/2
        current_snr = snr(data,mid)
    return mid

snrs=[1,3,5,7,10,15,20]

In [3]:
import librosa
import os
for root, dirs, files in os.walk("set"):
  for file in tqdm(files):
    if file.endswith(".wav"):
      the_file = f"{root}/{file}"
      y, s = librosa.load(the_file, sr=16000)
      sf.write(the_file, y, 16000)

100%|██████████| 150/150 [00:02<00:00, 62.24it/s]


In [2]:
import os 
from tqdm import tqdm 
for db in snrs:
    os.makedirs(f"set_{db}",exist_ok=True)

files = os.listdir("set")
for file in tqdm(files):
    y, sr = sf.read(f"set/{file}")
    for db in snrs:
        thres = determine_threshold(y, db)
        sf.write(f"set_{db}/{file}",clip(y,thres),sr)

100%|██████████| 150/150 [00:10<00:00, 14.34it/s]
